1. Load the csv data set of Twitter User labelled as bots with a filepath of "backend/mlops/data/raw/bot_tweets.csv"

In [252]:
import pandas as pd



def load_data()->pd.DataFrame:
    df = pd.read_csv("https://raw.githubusercontent.com/monoerea/x-bot-spt-ext/main/backend/mlops/data/raw/bot_tweets.csv")
    display(df.head(1))
    return df

df = load_data()

,uid,created_at,id,id_str,text,truncated,entities,extended_entities,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries
0,36741729,Fri Apr 07 20:16:57 +0000 2023,1.640000e+18,1.640000e+18,RT @TexasFootball: Had the whole offense hyped...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1644425327723126784, 'id_str...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,218,0,False,False,False,en,NaN,NaN,NaN,NaN


The columns or features of Tweets contain nested json values, and attached is a User Object.
1. Seperate the User Object into another csv file.
2. Flatten the json data to individual columns with appropriate column names.

In [253]:
import ast
user_df = df["user"].copy().apply(lambda x: ast.literal_eval(x))
user_df = pd.json_normalize(user_df)
display(user_df.head(1))

,id,id_str,name,screen_name,location,description,url,protected,followers_count,friends_count,...,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,withheld_in_countries,entities.description.urls,entities.url.urls
0,36741729,36741729,Matthew Stubblefield,Matthew_Brody,United States,Brody and Beckham’s Dad / All Gas. No Brakes. ...,None,False,264,410,...,True,False,False,False,False,False,none,[],[],NaN


In [254]:
def flatten(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """Flattens multiple JSON-like object columns at once."""
    json_data = pd.concat([pd.json_normalize(df[col]).add_prefix(f"{col}.") for col in cols], axis=1)

    return df.drop(columns=cols).reset_index(drop=True).join(json_data)

tweet_df = df[df.columns.difference(["user"])]

# Get object columns
obj_cols = tweet_df.select_dtypes(include=["object"]).columns.tolist()

# Flatten all JSON-like object columns in one step
tweet_df = flatten(tweet_df, obj_cols)

display(tweet_df.head(10))


,contributors,favorite_count,favorited,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,truncated,uid
0,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,218,False,False,36741729
1,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,341,False,False,36741729
2,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,74,False,False,36741729
3,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,49,False,False,36741729
4,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,3,False,False,36741729
5,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,1,False,False,36741729
6,NaN,0,False,1.640000e+18,1.640000e+18,1.640000e+18,1.640000e+18,74059488.0,74059488.0,False,NaN,NaN,0,False,False,36741729
7,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,True,1.640000e+18,1.640000e+18,5,False,False,36741729
8,NaN,0,False,1.640000e+18,1.640000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,25,False,False,36741729
9,NaN,0,False,1.630000e+18,1.630000e+18,NaN,NaN,NaN,NaN,False,NaN,NaN,4,False,False,36741729


As we can see, the tweets all have the same uid, but they are different tweets, the data was changed when opening and accidentally saving in Excel. However looking at the retweet count they are all different. Now that we have flatten the data, we can now properly draw insights from it.